In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, DateTime, Interval
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import alembic
import psycopg2
from dotenv import load_dotenv

In [ ]:
load_dotenv()

# Read environment variables
DB_USERNAME = os.environ['DB_USERNAME']
DB_PASSWORD = os.environ['DB_PASSWORD']
DB_HOST = os.environ['DB_HOST']
DB_PORT = os.environ['DB_PORT']
DB_NAME = os.environ['DB_NAME']

# Construct the database URL
DB_URL = f"postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DB_URL, echo=True)

In [30]:
Base = declarative_base()

In [ ]:
class TeamGameStats(Base):
    total = Column(Float)
    off_total = Column(Float)
    off_pass = Column(Float)
    off_rush = Column(Float)
    off_exp_turn = Column(Float)
    
    def_tot = Column(Float)
    def_pass = Column(Float)
    def_rush = Column(Float)
    def_exp_turn = Column(Float)
        
    first_downs = Column(Integer)
    rush_att = Column(Integer)
    rush_tds = Column(Integer)
    rush_yds = Column(Integer)
    
    pass_att = Column(Integer)
    pass_comp = Column(Integer)
    pass_yds = Column(Integer)
    pass_tds = Column(Integer)
    pass_int = Column(Integer)
    
    sack_yds = Column(Integer)
    total_yds = Column(Integer)
    fumbles = Column(Integer)
    fumbles_lost = Column(Integer)
    turnovers = Column(Integer)
    penalties = Column(Integer)
    penalty_yds = Column(Integer)
    
    time_of_pos = Column(Interval)

class Game(Base):
    away_team = Column(String)
    away_score = Column(Integer)
    home_team = Column(String)
    home_score = Column(Integer)

    roof = Column(String)
    surface = Column(String)
    Weather = Column(String)
    
    over_under = Column(Float)
    
    home_total = Column(Float)
    home_off_total = Column(Float)
    home_off_pass = Column(Float)
    home_off_rush = Column(Float)
    home_exp_turn = Column(Float)
    
    home_first_downs = Column(Integer)
    home_rush_att = Column(Integer)
    home_rush_tds = Column(Integer)
    home_rush_yds = Column(Integer)
    

In [3]:
class QbStatMixin:
    pass_first_downs = Column(Integer)
    pass_first_down_perc = Column(Float)
    pass_IAY = Column(Integer)
    pass_CAY = Column(Integer)
    pass_drops = Column(Integer)
    pass_bad_throw = Column(Integer)
    pass_bad_throw_perc = Column(Float)

    pass_pressure_perc = = Column(Float)
    pass_sacks = Column(Integer)
    pass_per_att = Column(Integer)
    pass_adj_per_att = Column(Integer)
    
class RbStatMixin:
    rush_first_downs = Column(Integer)
    rush_ybc = Column(Integer)
    rush_yac = Column(Integer)
    rush_brkTkl = Column(Integer)
    
    
class RecStatMixin:
    rec_first_downs = Column(Integer)
    rec_ybc = Column(Integer)
    rec_yac = Column(Integer)
    rec_adot = Column(Float)
    rec_brkTkl = Column(Integer)
    rec_drop = Column(Integer)
    rec_int = Column(Integer)

class PlayerStatMixin:
    # Passing
    pass_comp = Column(Integer)
    pass_att = Column(Integer)
    pass_yds = Column(Integer)
    pass_tds = Column(Integer)
    pass_ints = Column(Integer)

    # Rushing
    rush_att = Column(Integer)
    rush_yds = Column(Integer)
    rush_per_att = Column(Float)
    rush_tds = Column(Integer)
    
    # Receiving
    rec_tgt = Column(Integer)
    rec_rec = Column(Integer)
    rec_yds = Column(Integer)
    rec_per_rec = Column(Integer)
    rec_tds = Column(Integer)
    
    # Misc
    fumbles = Column(Integer)
    snaps = Column(Integer)
    games_played = Column(Integer)
    games_started = Column(Integer)
    snap_perc = Column(Float)

class Player(Base):
    __tablename__ = 'player'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    drafted_year = Column(DateTime)
    age = Column(Float)
    pos = Column(String)
    pos_alt = Column(String)
    
    season_stats = relationship("SeasonStats", back_populates="player")
    career_stats = relationship("CareerStats", back_populates="player")
    single_stats = relationship("SingleGameStats", back_populates="player")
    
class CareerStats(PlayerStatMixin, Base):
    __tablename__ = 'career_stats'
    id = Column(Integer, primary_key=True)
    player_id = Column(Integer, ForeignKey('player.id'))
    player = relationship("Player", back_populates="career_stats")


class SeasonStats(PlayerStatMixin, Base):
    __tablename__ = 'season_stats'
    id = Column(Integer, primary_key=True)
    year = Column(DateTime)
    player_id = Column(Integer, ForeignKey('player.id'))
    player = relationship("Player", back_populates="season_stats")
    
    team = Column(String)

class SingleGameStats(PlayerStatMixin, Base):
    __tablename__ = 'game_stats'
    id = Column(Integer, primary_key=True)
    week = Column(Integer)
    
    player_id = Column(Integer, ForeignKey('player.id'))
    player = relationship("Player", back_populates="game_stats")
    
    season_id = Column(Integer, ForeignKey('season_stats.id'))
    season = relationship("SeasonStats", back_populates="game_stats")
    
    team = Column(String)
    team_vs = Column(String)

NameError: name 'Base' is not defined

In [ ]:
class AggregateSeasonStats(PlayerStatMixin, Base):
    __tablename__ = 'agg_season_stats'
    id = Column(Integer, primary_key=True)
    
    year = Column(DateTime)